# Data Ingest of 10-h Fuel Moisture Content

This notebook demonstrates retrieval and filtering of 10-h dead FMC data from a RAWS stash maintained by Angel Farguell.

## References

For more info on python library API, see Brian Blaylock's `SynopticPy` [python package](https://github.com/blaylockbk/SynopticPy)

For more info on available Synoptic RAWS variables, see [Synoptic Data](https://demos.synopticdata.com/variables/index.html) documentation

## Setup

In [ ]:
import os.path as osp
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import synoptic
import json
import sys
import numpy as np
import polars as pl
import pandas as pd
sys.path.append('../src')
from utils import Dict, read_yml, read_pkl, str2time
from data_funcs import rename_dict
import ingest.retrieve_raws_api as rr
import ingest.retrieve_raws_stash as rrs

In [ ]:
# raws_meta = read_yml("../etc/variable_metadata/raws_metadata.yaml")

with open("../etc/training_data_config.json", "r") as json_file:
    config = json.load(json_file)   
    config = Dict(config)

In [ ]:
config

In [ ]:
raws_stash_path = rrs.raws_meta["raws_stash_path"]
print(raws_stash_path)
osp.exists(raws_stash_path)

In [ ]:
start = str2time(config.start_time)
end  = str2time(config.end_time)
bbox = config.bbox
bbox_reordered = [bbox[1], bbox[0], bbox[3], bbox[2]]

In [ ]:
start

In [ ]:
start.year

In [ ]:
sts = rr.get_stations(bbox_reordered, start, end)
sts

In [ ]:
# times = pd.date_range(start-relativedelta(hours=1), end, freq="h")
# times = times.to_pydatetime()
# times

In [ ]:
# def get_file_paths(start, end):
#     """
#     Get file paths for RAWS stash from given start and end dates.
#     """  
#     # yr = str(time.year)
#     # doy = time.strftime('%j') # Julian day of year, 001-366
#     # hod = time.strftime('%H') # Hour of day, 00-23

#     paths = [
#         osp.join(
#             raws_stash_path,        
#             str(time.year),     
#             time.strftime('%j'),    #  Julian day of year, 001-366
#             f"{str(time.year)}{time.strftime('%j')}{time.strftime('%H')}.pkl" )# Join with hour of day, 00-23
#         for time in times
#     ]
    
#     return paths

In [ ]:
paths = rrs.get_file_paths(start, end)
paths

In [ ]:
np.all([osp.exists(path) for path in paths])

In [ ]:
sts

In [ ]:
# times

In [ ]:
paths

In [ ]:
d1 = read_pkl(paths[0])
d1

In [ ]:
sts[0]

In [ ]:
# name_mapping = {
#     "STID": "stid",
#     "datetime": "date_time",
#     "fm10": "fm",
#     "latitude": "lat",
#     "longitude": "lon",
#     "elevation": "elev"
# }


In [ ]:
# def get_static(df, st, static_vars=["stid", "latitude", "longitude", "elevation", "name", "state", "id"]):
#     loc = {col: values[0] for col, values in sts.filter(sts["stid"] == st).select(static_vars).to_dict(as_series=False).items()}
#     loc = rename_dict(loc, name_mapping)
#     loc["elev"] = loc["elev"] * 0.3048 # Convert ft to M
    
#     return loc

In [ ]:
rr.get_static(sts, "BRLW4")

In [ ]:
raws_dict = {
    st: {
        "RAWS": [],
        "units": {"fm": "%", "elev": "m"},
        "loc": rr.get_static(sts, st),
        "misc": "Data collected from RAWS stash"
        } 
    for st in sts["stid"]}
raws_dict

In [ ]:
raws_dict.keys()

In [ ]:
raws_dict["TS399"]["loc"]

In [ ]:
for path in paths:
    dat = read_pkl(path)
    for st in sts["stid"]:
        # Filter the data for the current station and append
        filtered = dat[dat['STID'] == st]
        if not filtered.empty:
            raws_dict[st]["RAWS"].append(filtered)

In [ ]:
# Combine the lists of DataFrames for each station into a single DataFrame

for st in raws_dict:
    if raws_dict[st]["RAWS"]:  # Check if the list is not empty
        raws_dict[st]["RAWS"] = pd.concat(raws_dict[st]["RAWS"], ignore_index=True)
    else:
        raws_dict[st]["RAWS"] = pd.DataFrame()  # Set an empty DataFrame if no data was found
    raws_dict[st]["RAWS"].rename(columns=rrs.raws_meta["rename_stash"], inplace=True)

In [ ]:
raws_dict["BRLW4"]["RAWS"]

In [ ]:
type(raws_dict["BRLW4"]["RAWS"])

In [ ]:
read_pkl(osp.join(raws_stash_path, "stations.pkl"))

In [ ]:
raws_dict["BRLW4"].keys()

In [ ]:
raws_dict["BRLW4"]["misc"]

In [ ]:
raws_dict["BRLW4"]["loc"]

In [ ]:
raws_dict["BRLW4"]["units"]

In [ ]:
sts

In [ ]:
from herbie import HerbieLatest

In [ ]:
ds = HerbieLatest(model="hrrr", product="prs", fxx=0).terrain()

In [ ]:
ds

In [ ]:
points = pd.DataFrame(sts[["longitude", "latitude"]], columns=["longitude", "latitude"])
points

In [ ]:
points.shape

In [ ]:
ds_pts = ds.herbie.pick_points(points, method="nearest")
ds_pts

In [ ]:
from herbie.toolbox import EasyMap, pc
from herbie import paint

In [ ]:
ax = EasyMap("50m", crs=ds.herbie.crs, figsize=[10, 8]).BORDERS().STATES().ax

p = ax.pcolormesh(
    ds.longitude,
    ds.latitude,
    ds.orog,
    transform=pc,
    cmap = paint.LandGreen.cmap,
)

In [ ]:
# ds_pts = ds_pts.expand_dims({"stid": sts["stid"].to_numpy()})
ds_pts = ds_pts.assign_coords(point = sts["stid"].to_numpy())

In [ ]:
ds_pts

In [ ]:
ax = EasyMap("50m", crs=ds.herbie.crs, figsize=[10, 8]).BORDERS().STATES().ax

p = ax.pcolormesh(
    ds.longitude,
    ds.latitude,
    ds.orog,
    transform=pc,
    cmap = paint.LandGreen.cmap,
)

for i in ds_pts.point:
    z = ds_pts.sel(point=i)
    ax.scatter(z.longitude, z.latitude, color="k", transform=pc)
    ax.scatter(
        z.point_longitude, z.point_latitude, color="blue", marker=".", transform=pc
    )

In [ ]:
ax = EasyMap("50m", crs=ds.herbie.crs, figsize=[10, 8]).BORDERS().STATES().ax

p = ax.pcolormesh(
    ds.longitude,
    ds.latitude,
    ds.orog,
    transform=pc,
    cmap = paint.LandGreen.cmap,
)

for i in ds_pts.point:
    z = ds_pts.sel(point=i)
    ax.scatter(z.longitude, z.latitude, color="k", transform=pc)
    ax.scatter(
        z.point_longitude, z.point_latitude, color="blue", marker=".", transform=pc
    )
    ax.text(
        z.point_longitude,
        z.point_latitude,
        f" {z.point.item()}",
        transform=pc,
    )

buff = 1
ax.set_extent([bbox[1]-buff, bbox[3]+buff, bbox[0]-buff, bbox[2]+buff])